In [1]:
import pickle
import pandas as pd
import d6tstack
import numpy as np
%cd ..


/home/dyllanjr/Documents/Classification-M2/notebooks


In [2]:
from pyfuncs import processing

In [3]:
%cd /home/dyllanjr/Documents/Classification-M2/notebooks/models/


/home/dyllanjr/Documents/Classification-M2/notebooks/models


In [4]:
model = pickle.load(open('forest_sample_trained.pkl', 'rb'))

In [41]:
def sensor_events(row):
    '''Converts sensor instance to number and returns list of lists of sensor events'''
    output = []
    convention = {'sensorCount-Bathroom': 1,
             'sensorCount-Bedroom': 2, 
             'sensorCount-Chair': 3,
             'sensorCount-DiningRoom': 4, 
             'sensorCount-Hall': 5, 
             'sensorCount-Kitchen':6, 
             'sensorCount-LivingRoom': 7,
            'sensorCount-Office': 8, 
            'sensorCount-OutsideDoor': 9,
            'sensorCount-WorkArea': 10}
    for col, value in convention.items():
        if row[col] > 0:
            output.append(value)
    if output == []:
        output.append(0)
    return output

In [42]:
def load_predictions(model):
    '''Loads ypred, ytrue and sensor events into psql db'''
    csvs = np.arange(101, 111)
    #unsampled csh's are 101, 105, 110
    del_csvs = np.array([101, 105, 110])
    csvs = np.setdiff1d(csvs, del_csvs)
    for csv in csvs:
        X, y =  processing.load_house(csv)
        X = X.iloc[100100::] #have a bit of spacing between train and test set where possible data leakage
        y = y.iloc[100100::] #this is extracting the test set of the houses we modelled off of
        ypred = model.predict(X)
        sensors_data = X.apply(lambda row: sensor_events(row), axis=1)
        df = pd.DataFrame({'true': y, 'pred': ypred, 'sensors': sensors_data})
        username = 'postgres'  # DB username
        password = 'djru'  # DB password
        host = 'localhost'  # Public IP address for your instance
        port = '5432'
        database = 'class_prediction_api'  # Name of database ('postgres' by default)
        db_uri = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(username, password, host, port, database)
        d6tstack.utils.pd_to_psql(df.reset_index(), db_uri, 'csh' + str(csv), sep='\t', if_exists='replace')
    del X, y, ypred, df, sensors

In [43]:
load_predictions(model)

UnboundLocalError: local variable 'sensors' referenced before assignment

In [10]:
X, y = processing.load_house(110)

In [35]:
X.reset_index()

,index,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,...,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
0,0,3.0,11131.0,2.0,10942.0,590.0,0.0,0.0,1.0,1.0,...,0.000000,86400.0,86400.000000,86400.0,590.491697,86400.000000,86400.000000,86400.0,86400.000000,86400.0
1,1,3.0,11132.0,2.0,10068.0,1.0,0.0,0.0,1.0,1.0,...,0.000000,86400.0,86400.000000,86400.0,591.597136,86400.000000,86400.000000,86400.0,86400.000000,86400.0
2,2,3.0,11141.0,2.0,9752.0,9.0,0.0,0.0,5.0,5.0,...,8.311974,86400.0,86400.000000,86400.0,0.000000,86400.000000,86400.000000,86400.0,86400.000000,86400.0
3,3,3.0,11588.0,2.0,9324.0,447.0,0.0,0.0,5.0,5.0,...,456.077885,86400.0,86400.000000,86400.0,0.000000,86400.000000,86400.000000,86400.0,86400.000000,86400.0
4,4,3.0,11740.0,2.0,9151.0,152.0,0.0,0.0,5.0,5.0,...,608.264106,86400.0,86400.000000,86400.0,0.000000,86400.000000,86400.000000,86400.0,86400.000000,86400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136711,136711,11.0,40935.0,4.0,542.0,17.0,5.0,5.0,5.0,5.0,...,7136.841028,86400.0,17.515269,86400.0,0.000000,5640.581092,7118.620603,86400.0,7132.380529,86400.0
136712,136712,11.0,40993.0,4.0,596.0,58.0,5.0,5.0,5.0,5.0,...,7195.091198,86400.0,75.765439,86400.0,0.000000,5698.831262,7176.870773,86400.0,7190.630699,86400.0
136713,136713,11.0,41001.0,4.0,603.0,8.0,5.0,5.0,5.0,5.0,...,7203.117300,86400.0,83.791541,86400.0,0.000000,5706.857364,7184.896875,86400.0,7198.656801,86400.0
136714,136714,11.0,41003.0,4.0,604.0,2.0,5.0,5.0,5.0,5.0,...,7204.979627,86400.0,85.653868,86400.0,0.000000,5708.719691,7186.759202,86400.0,7200.519128,86400.0


In [37]:
test = X.apply(lambda row: sensor_events(row), axis=1)

In [15]:
type(test)

pandas.core.series.Series

In [12]:
pd.Series(test).value_counts()

[2]                39469
[1, 2]             12717
[6]                10573
[1]                 8532
[6, 7]              6058
                   ...  
[1, 2, 3, 7]          10
[2, 4, 9]              9
[4, 6, 9]              6
[1, 4]                 4
[3, 4, 6, 7, 9]        1
Length: 78, dtype: int64

In [38]:
ypred = model.predict(X)

In [39]:
df = pd.DataFrame({'true': y, 'pred': ypred, 'sensors': test}).reset_index()

In [40]:
df.head()

,index,true,pred,sensors
0,0,Sleep,Sleep,{2}
1,1,Sleep,Sleep,{2}
2,2,Sleep,Sleep,{2}
3,3,Sleep,Sleep,{2}
4,4,Sleep,Sleep,{2}


In [17]:
df.sensors.astype()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136716 entries, 0 to 136715
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   true     136716 non-null  object
 1   pred     136716 non-null  object
 2   sensors  136716 non-null  object
dtypes: object(3)
memory usage: 3.1+ MB


In [ ]:
pd.options.display.max_columns =100

In [56]:
X.iloc[10000:10010]

,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,complexity,activityChange,areaTransitions,numDistinctSensors,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
10000,8.0,30161.0,3.0,72.0,7.0,1.0,1.0,0.0,0.0,0.0,1.362898,0.388889,2.0,0.0,19.62,8.62,0.0,0.0,0.0,6.11,0.0,0.0,0.0,0.0,0.0,0.000000,56.877522,58605.233552,51491.702347,86400.0,38.337099,26630.091341,26610.764166,86400.0,26603.699219,86400.0
10001,8.0,30162.0,3.0,73.0,1.0,1.0,1.0,0.0,0.0,0.0,1.294820,0.410959,2.0,0.0,20.80,8.69,0.0,0.0,0.0,4.86,0.0,0.0,0.0,0.0,0.0,0.000000,57.409088,58605.765118,51492.233913,86400.0,38.868665,26630.622907,26611.295732,86400.0,26604.230785,86400.0
10002,8.0,30163.0,3.0,72.0,1.0,1.0,1.0,0.0,0.0,0.0,1.241946,0.416667,2.0,0.0,21.99,7.46,0.0,0.0,0.0,4.90,0.0,0.0,0.0,0.0,0.0,0.000000,58.531268,58606.887298,51493.356093,86400.0,39.990845,26631.745087,26612.417912,86400.0,26605.352965,86400.0
10003,8.0,30163.0,3.0,72.0,0.0,1.0,1.0,0.0,0.0,0.0,1.156780,0.444444,2.0,0.0,23.19,7.52,0.0,0.0,0.0,3.64,0.0,0.0,0.0,0.0,0.0,0.000000,59.293342,58607.649372,51494.118167,86400.0,40.752919,26632.507161,26613.179986,86400.0,26606.115039,86400.0
10004,8.0,30167.0,3.0,74.0,4.0,1.0,1.0,0.0,0.0,0.0,1.091156,0.432432,2.0,0.0,24.40,6.28,0.0,0.0,0.0,3.67,0.0,0.0,0.0,0.0,0.0,0.000000,63.079965,58611.435995,51497.904790,86400.0,44.539542,26636.293784,26616.966609,86400.0,26609.901662,86400.0
10005,8.0,30168.0,3.0,75.0,1.0,1.0,1.0,0.0,0.0,0.0,0.985172,0.453333,2.0,0.0,25.62,6.33,0.0,0.0,0.0,2.40,0.0,0.0,0.0,0.0,0.0,0.000000,63.622193,58611.978223,51498.447018,86400.0,45.081770,26636.836012,26617.508837,86400.0,26610.443890,86400.0
10006,8.0,30174.0,3.0,74.0,6.0,1.0,1.0,5.0,5.0,0.0,1.013664,0.418919,2.0,0.0,25.85,5.08,0.0,0.0,0.0,3.42,0.0,0.0,0.0,0.0,0.0,6.336706,69.958899,58618.314929,51504.783724,86400.0,0.000000,26643.172718,26623.845543,86400.0,26616.780596,86400.0
10007,8.0,30176.0,3.0,75.0,2.0,1.0,1.0,0.0,0.0,0.0,0.921928,0.413333,2.0,0.0,27.08,3.82,0.0,0.0,0.0,3.45,0.0,0.0,0.0,0.0,0.0,0.000000,71.462041,58619.818071,51506.286866,86400.0,1.503142,26644.675860,26625.348685,86400.0,26618.283738,86400.0
10008,8.0,30176.0,3.0,74.0,0.0,1.0,1.0,0.0,0.0,0.0,0.811848,0.445946,2.0,0.0,28.32,2.55,0.0,0.0,0.0,3.48,0.0,0.0,0.0,0.0,0.0,0.000000,72.065182,58620.421212,51506.890007,86400.0,2.106283,26645.279001,26625.951826,86400.0,26618.886879,86400.0
10009,8.0,30178.0,3.0,75.0,2.0,1.0,1.0,0.0,0.0,0.0,0.511117,0.440000,1.0,0.0,29.57,1.27,0.0,0.0,0.0,3.51,0.0,0.0,0.0,0.0,0.0,0.000000,74.295184,58622.651214,51509.120009,86400.0,4.336285,26647.509003,26628.181828,86400.0,26621.116881,86400.0


In [55]:
y.iloc[10000:10010]

10000    Personal_Hygeine
10001    Personal_Hygeine
10002    Personal_Hygeine
10003    Personal_Hygeine
10004    Personal_Hygeine
10005    Personal_Hygeine
10006    Personal_Hygeine
10007    Personal_Hygeine
10008    Personal_Hygeine
10009    Personal_Hygeine
Name: reduced_activity, dtype: object